# Tutorial: Columnstore indexes

This notebook uses the sample AdventureWorks sample database but can be used with any user database.

# Step 1: Enable Always Encrypted with secure enclaves in SQL Server

## As a first step let us create a tablee which is large enough to show performance impact of columnstore index. The time taken to create sample tablemay vary on different computer based on the resources.

In [ ]:
USE AdventureWorks
GO
-- Create New Table
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[Sales].[BigSalesOrderDetail]') AND type in (N'U'))
DROP TABLE [Sales].[BigSalesOrderDetail]
GO
CREATE TABLE [Sales].[BigSalesOrderDetail](
[SalesOrderID] [int] NOT NULL,
[SalesOrderDetailID] [int] NOT NULL,
[CarrierTrackingNumber] [nvarchar](25) NULL,
[OrderQty] [smallint] NOT NULL,
[ProductID] [int] NOT NULL,
[SpecialOfferID] [int] NOT NULL,
[UnitPrice] [money] NOT NULL,
[UnitPriceDiscount] [money] NOT NULL,
[LineTotal] [numeric](38, 6) NOT NULL,
[rowguid] [uniqueidentifier] NOT NULL,
[ModifiedDate] [datetime] NOT NULL
) ON [PRIMARY]
GO
-- Create clustered index
CREATE CLUSTERED INDEX [CL_BigSalesOrderDetail] ON [Sales].[BigSalesOrderDetail]
( [SalesOrderDetailID])
GO
-- Create Sample Data Table
-- WARNING: This Query may run upto 2-10 minutes based on your systems resources
INSERT INTO [Sales].[BigSalesOrderDetail]
SELECT S1.*
FROM Sales.SalesOrderDetail S1
GO 100

# Step 2: Run the query and observe IO statistics

In [ ]:
-- Performance Test
-- Comparing Regular Index with ColumnStore Index
USE AdventureWorks
GO
SET STATISTICS IO ON
GO
-- Select Table with regular Index
SELECT ProductID, SUM(UnitPrice) SumUnitPrice, AVG(UnitPrice) AvgUnitPrice,
SUM(OrderQty) SumOrderQty, AVG(OrderQty) AvgOrderQty
FROM [Sales].[BigSalesOrderDetail]
GROUP BY ProductID
ORDER BY ProductID
GO
-- Table 'BigSalesOrderDetail'. Scan count 1, logical reads 342261, physical reads 0, read-ahead reads 0.

# Step 3: Create a Columnstore index

In [ ]:
-- Create ColumnStore Index
CREATE NONCLUSTERED COLUMNSTORE INDEX [IX_BigSalesOrderDetail_ColumnStore]
ON [BigSalesOrderDetail]
(UnitPrice, OrderQty, ProductID)
GO

# Step 4: Run the same query now with the columnstore index

In [ ]:
-- Select Table with Columnstore Index
SELECT ProductID, SUM(UnitPrice) SumUnitPrice, AVG(UnitPrice) AvgUnitPrice,
SUM(OrderQty) SumOrderQty, AVG(OrderQty) AvgOrderQty
FROM [Sales].[BigSalesOrderDetail]
GROUP BY ProductID
ORDER BY ProductID
GO

3\. Try the same query again in the SSMS instance that doesn't have Always Encrypted enabled, and note the failure that occurs.